In [3]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

# Função para extrair dados de um cupom XML individual
def extrair_dados_cupom(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Lista para armazenar os dados de cada item da compra
    lista_itens = []

    # Extraindo informações do cliente
    dest = root.find(".//dest")
    cpf_cliente = dest.findtext("CPF") if dest is not None else None
    nome_cliente = dest.findtext("xNome") if dest is not None else None

    # Extraindo informações gerais do cupom
    ide = root.find(".//ide")
    numero_cupom = ide.findtext("nCFe") if ide is not None else None
    data_emissao = ide.findtext("dEmi") if ide is not None else None
    hora_emissao = ide.findtext("hEmi") if ide is not None else None

    # Corrigindo o formato da data e hora
    try:
        data_emissao_formatada = datetime.strptime(data_emissao, "%Y%m%d").strftime("%d/%m/%Y") if data_emissao else None
    except ValueError:
        data_emissao_formatada = None

    try:
        hora_emissao_formatada = datetime.strptime(hora_emissao, "%H%M%S").strftime("%H:%M") if hora_emissao else None
    except ValueError:
        hora_emissao_formatada = None

    # Extraindo valores da compra corretamente
    vCFe = root.findtext(".//total/vCFe", default="0.00")  # Valor total da compra
    vDesc = root.findtext(".//total/ICMSTot/vDesc", default="0.00")  # Desconto
    vTrib = root.findtext(".//total/vCFeLei12741", default="0.00")  # Valor total dos tributos

    # Extraindo meio de pagamento
    pagto = root.find(".//MP")
    meio_pagamento = pagto.findtext("cMP") if pagto is not None else None  # Código do meio de pagamento

    # Extraindo produtos (itens do cupom)
    for det in root.findall(".//det"):
        ean = det.findtext(".//cEAN", default="")  # Obtendo o EAN
        codigo_produto = det.findtext(".//cProd")
        nome_produto = det.findtext(".//xProd", default="")

        # Pegando a descrição do produto (texto após o último "#")
        descricao = nome_produto.split("#")[-1].strip() if "#" in nome_produto else nome_produto

        item = {
            "Número do Cupom": numero_cupom,
            "Data da Emissão": data_emissao_formatada,
            "Hora da Emissão": hora_emissao_formatada,
            "CPF Cliente": cpf_cliente,
            "Nome Cliente": nome_cliente,
            "EAN": ean,  
            "Código Produto": codigo_produto,
            "Descrição Produto": descricao,
            "Quantidade": float(det.findtext(".//qCom", default="0.00")),
            "Valor Unitário": float(det.findtext(".//vUnCom", default="0.00")),
            "Valor Total Produto": float(det.findtext(".//vProd", default="0.00")),
            "Total Tributos (Lei 12.741/12)": float(vTrib)
        }
        lista_itens.append(item)

    # Adicionando os valores totais e forma de pagamento a cada item do cupom
    for item in lista_itens:
        item["Valor Total Compra"] = float(vCFe) if vCFe else 0.00
        item["Desconto"] = float(vDesc) if vDesc else 0.00
        item["Meio de Pagamento"] = meio_pagamento

    return lista_itens

# Função para processar múltiplos arquivos XML de um diretório
def processar_cupons(diretorio):
    lista_dados = []

    for arquivo in os.listdir(diretorio):
        if arquivo.endswith(".xml"):
            caminho_arquivo = os.path.join(diretorio, arquivo)
            lista_dados.extend(extrair_dados_cupom(caminho_arquivo))

    # Criando DataFrame do Pandas
    df = pd.DataFrame(lista_dados)
    return df

# === Processamento e salvamento ===
diretorio_xmls = "C:/Users/User01/arquivos_xml" # Defina o diretório onde os cupons XML serão coletados e o CSV será salvo.
df_cupons = processar_cupons(diretorio_xmls)

# Salvar CSV no diretório definido com codificação correta para Excel
df_cupons.to_csv(os.path.join(diretorio_xmls, "dados_de_compras.csv"), index=False, encoding="utf-8-sig")

# Exibir primeiras linhas
print(df_cupons.head())

  Número do Cupom Data da Emissão Hora da Emissão  CPF Cliente  \
0          009957      02/01/2024           10:57  06861835846   
1          009957      02/01/2024           10:57  06861835846   
2          009958      02/01/2024           11:45  22196592838   
3          009959      02/01/2024           12:06  43536323880   
4          009960      02/01/2024           12:12  45154657837   

                        Nome Cliente            EAN Código Produto  \
0                 Vitoria dos Santos  7896986266010        1002836   
1                 Vitoria dos Santos  7896986260155        1002538   
2  Marcio Diego Marciano de Oliveira  7896986263064        2003540   
3             Elisete Lopes de Jesus  7896986261459        1002602   
4          Andressa Ferreira da Siva  7896986283291        1003186   

    Descrição Produto  Quantidade  Valor Unitário  Valor Total Produto  \
0      SORV DR MOR 70         1.0            13.9                 13.9   
1     SORV COOKIE  70         1.0 